In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from PIL import Image
import pandas as pd
import os
import torch.optim.lr_scheduler as lr_scheduler  # 导入学习率调整器

# 检查CUDA设备是否可用，选择设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 数据路径
train_dir = "F:\\python代码\\机器学习\\期末作业\\train"
pred_dir = "F:\\python代码\\机器学习\\期末作业\\pred"
csv_dir = "F:\\python代码\\机器学习\\期末作业\\pre_data.csv"
# 标签字典
label_dict = {0: 'cavallo', 1: 'mucca', 2: 'scoiattolo'}

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),  # 添加水平翻转
    transforms.RandomRotation(15),  # 添加随机旋转
    transforms.ToTensor(),

])

# 加载数据集
train_data = datasets.ImageFolder(train_dir, transform=transform)
# 划分数据集
train_size = int(0.7 * len(train_data))
val_size = int(0.15 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(train_data, [train_size, val_size, test_size])

# 数据加载器
batch_size = 16  # 增加批量大小
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        # 保存残差
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        # 匹配维度
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=3):
        super(ResNet, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet layers
        self.layer1 = self._make_layer(block, 64, num_block[0], 1)
        self.layer2 = self._make_layer(block, 128, num_block[1], 2)
        self.layer3 = self._make_layer(block, 256, num_block[2], 2)
        self.layer4 = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.pool(output)
        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)
        output = self.layer4(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output

# 定义ResNet-18
def resnet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

model = resnet18(num_classes=3).to(device) 

Using device: cuda:0


In [12]:
import os
from torchvision import transforms
from PIL import Image
import csv

model_dir="model_weights.pth"
#修改这里验收位置
pred_dir = "F:\\python代码\\机器学习\\期末作业\\pred"
# 创建用于预测的数据集
class PredictionDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = os.listdir(folder_path)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.images[idx])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.images[idx]

# 预测数据集的数据预处理
pred_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 创建用于预测的数据集
pred_dataset = PredictionDataset(folder_path=pred_dir, transform=pred_transform)
pred_loader = DataLoader(pred_dataset, batch_size=1, shuffle=False)



# 加载保存的权重
loaded_model = resnet18(num_classes=3).to(device)
loaded_model.load_state_dict(torch.load(model_dir))
loaded_model.eval()

# 预测并输出结果
loaded_predictions = []

with torch.no_grad():
    for inputs, filenames in pred_loader:
        inputs = inputs.to(device)
        outputs = loaded_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        loaded_predictions.append((filenames[0], predicted.item()))

# 输出结果到 pred.csv 文件
output_file_loaded = "pred_YANSHOU.csv"

with open(output_file_loaded, mode='w', newline='') as csvfile:
    fieldnames = ['Image Name', 'Label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for filename, prediction in loaded_predictions:
        writer.writerow({'Image Name': filename, 'Label': prediction})

print(f"Predictions from loaded model have been written to {output_file_loaded}.")



Predictions from loaded model have been written to pred_YANSHOU.csv.
